## NECESSARY INSTALLS

In [ ]:
!pip install biosppy
!pip install opencv-python
!pip install gdown

In [ ]:
!ls

In [ ]:
# import necessary modules

import pandas as pd
import numpy as np
np.random.seed(42)

import matplotlib.pyplot as plt
import os
import sys
import pickle
import cv2
import tqdm
import biosppy
import scipy.signal as sig
from scipy.signal import spectrogram, windows, check_COLA
from scipy.stats import zscore
%matplotlib inline
import matplotlib as mlp
mlp.rc("xtick",labelsize=10)
mlp.rc("ytick",labelsize=10)
mlp.rc("axes",labelsize=11)
plt.rcParams["figure.figsize"] = [5,3]
plt.rcParams["figure.dpi"] = 100

CURR_DIR = os.getcwd()

%matplotlib inline
import matplotlib as mlp
mlp.rc("xtick",labelsize=10)
mlp.rc("ytick",labelsize=10)
mlp.rc("axes",labelsize=11)
plt.rcParams["figure.figsize"] = [8,5]
plt.rcParams["figure.dpi"] = 100

# a function to save plotted figures
def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=100):
    path = fig_id + "." + fig_extension
    print("Saving figure", fig_id)
#     if tight_layout:
#         plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution,pad_inches=-0.5,bbox_inches="tight")
    

In [ ]:

# hardware definitions

SAMPLING_RATE = 250 #Hz
NUM_CHANNELS = 8 
FONT_PATH = "../input/nepali-fonts/mangal.ttf"
font = mlp.font_manager.FontProperties(fname=FONT_PATH)
FIG_DIR = ""
# dataset definitions

SENTENCES =["अबको समय सुनाउ","एउटा सङ्गित बजाउ","आजको मौसम बताउ","बत्तिको अवस्था बदल","पङ्खाको स्तिथी बदल"]
LABELS = np.array(SENTENCES)[[3, 2, 1, 4, 0, 3, 2, 1, 4, 0, 3, 2, 1, 4, 0]]

CLASSES = {0:"अबको समय सुनाउ",
          1:"आजको मौसम बताउ",
          2:"एउटा सङ्गित बजाउ",
          3:"पङ्खाको स्तिथी बदल",
          4:"बत्तिको अवस्था बदल"}


<!-- 
CLASS | LABELS
--- | ---  
0 | "अबको समय सुनाउ",
1 | "आजको मौसम बताउ",
2 | "एउटा सङ्गित बजाउ",
3 | "पङ्खाको स्तिथी बदल",
4 | "बत्तिको अवस्था बदल" -->

## IMPORT DATASET

In [ ]:
TRAIN_PATH = "../input/emg-dataset/train(aug_filtered).pickle"
TEST_PATH = "../input/emg-dataset/test(unaug_filtered).pickle"
VAL_PATH = "../input/emg-dataset/val(unaug_filtered).pickle"

In [ ]:
print("Fetching data from pickle file ...")
train = pickle.load(open(TRAIN_PATH,"rb"))
test = pickle.load(open(TEST_PATH,"rb"))
val = pickle.load(open(VAL_PATH,"rb"))


# all_data_filtered = pickle.load(open(DATA_PATH,"rb"))
# print("Fetching raw data from pickle file ...")
# all_data = pickle.load(open(DATA_PATH,"rb"))
print("Done!")

In [ ]:
print(type(train))
print(train.keys())
print([len(x[-1]) for x in train.items()])
print(np.array(train["data"]).shape)

print(type(test))
print(test.keys())
print([len(x[-1]) for x in test.items()])
print(np.array(test["data"]).shape)

print(type(val))
print(val.keys())
print([len(x[-1]) for x in val.items()])
print(np.array(val["data"]).shape)

In [ ]:
df = pd.Series(train["labels"][:2119])
print(df.value_counts())
df = pd.Series(train["labels"][2119:4238])
print(df.value_counts())
df = pd.Series(train["labels"][-2119:])
print(df.value_counts())

In [ ]:
encoder = LabelEncoder()
encoded_labels = encoder.fit_transform(train["labels"][:2119].T).astype('float64')
X_train,y_train = np.array(train["data"][:2119]),encoded_labels
encoder.classes_

In [ ]:
encoded_labels = encoder.fit_transform(test["labels"].T).astype('float64')
X_test,y_test = np.array(test["data"]),encoded_labels
encoder.classes_

In [ ]:
encoded_labels = encoder.fit_transform(val["labels"].T).astype('float64')
X_val,y_val = np.array(val["data"]),encoded_labels
encoder.classes_

In [ ]:
print(X_train.shape)
print(y_train.shape)

print(X_test.shape)
print(y_test.shape)

print(X_val.shape)
print(y_val.shape)
# print(y_val[:5])
# encoder.classes_

In [ ]:
plt.plot(X_train[0])
plt.show()

In [ ]:
plt.plot(X_train[-1])
plt.show()

## PREPARE DATASET

In [ ]:
def feature_pipeline(data):
    feature_data = []
    def getSpect(sdata):
        M = 60
        win = windows.hann(M, sym=False)
        return spectrogram(x=np.array(sdata), fs=SAMPLING_RATE, window=win, nperseg=len(win), noverlap=3*M/4, nfft=M)
    def process_signal(data):
        f_data = []
        for i in range(8):
            _, _, c_data = getSpect(data[:, i])
            f_data.append(c_data)
        return np.array(f_data)
    
    for d in tqdm.tqdm(data,desc="EXTRACTING DATA: "):
        temp_data = process_signal(d)
        feature_data.extend([temp_data])
    return np.array(feature_data)

In [ ]:
# def feature_pipeline(data):
#     feature_data = []
#     def getSpect(sdata):
#         M = 60
#         win = windows.hann(M, sym=False)
#         return spectrogram(x=np.array(sdata), fs=SAMPLING_RATE, window=win, nperseg=len(win), noverlap=3*M/4, nfft=M)
#     def process_signal(data):
#         f_data = []
#         for i in range(8):
#             _, _, c_data = getSpect(data[:, i])
#             f_data.append(c_data.T)
#         return np.array(f_data).T
    
#     for d in tqdm.tqdm(data,desc="EXTRACTING DATA: "):
#         temp_data = process_signal(d)
#         feature_data.extend([temp_data])
#     return np.array(feature_data)

In [ ]:
X_train = feature_pipeline(X_train)
X_test = feature_pipeline(X_test)
X_val = feature_pipeline(X_val)

In [ ]:
print(X_train.shape)
print(y_train.shape)

print(X_test.shape)
print(y_test.shape)

print(X_val.shape)
print(y_val.shape)

In [ ]:
image = X_train[0][0]
fig = plt.figure(figsize=(5,2))
plt.pcolormesh(image, shading='gouraud',cmap="Greys")
plt.ylim([0,3])
plt.axis("off")
# save_fig(path)
# plt.close(fig)
plt.show()

## Save Spectograms As Images

In [ ]:
!rm -rf trai* tes* val*
for i in CLASSES.items():
    os.makedirs("train/"+i[-1],exist_ok=True)
    os.makedirs("test/"+i[-1],exist_ok=True)
    os.makedirs("val/"+i[-1],exist_ok=True)


In [ ]:
# a function to save plotted figures
def save_fig(fig_id, tight_layout=True, fig_extension="jpg", resolution=80):
    path = fig_id + "." + fig_extension
#     print("Saving figure", fig_id)
#     if tight_layout:
#         plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution,pad_inches=0.0,bbox_inches="tight")

def spec2image(X,Y,DIR):
    spec_image_all = []
    def save_image(image,path):
        fig = plt.figure(figsize=(5,2))
        plt.pcolormesh(image, shading='gouraud',cmap="Greys")
        plt.ylim([0,3])
        plt.axis("off")
        save_fig(path)
        plt.close(fig)
        plt.show()
        
    for i in tqdm.tqdm(range(len(X))):
        spec_image = np.zeros((120,310,8))
        for c in range(8):
            path = os.path.join(DIR,CLASSES[int(Y[i])],str(i)+"_channel_"+str(c))
            save_image(X[i][c],path)
            spec_image[:,:,c] = cv2.imread(path+".jpg",cv2.IMREAD_GRAYSCALE)
            os.remove(path+".jpg")
        spec_image_all.append(spec_image)
    return np.array(spec_image_all)

In [ ]:
# train = spec2image(X_train,y_train,DIR="train")
# test  = spec2image(X_test,y_test,DIR="test")
# val = spec2image(X_val,y_val,DIR="val")

In [ ]:
# pickle.dump(val,open("val_spec_image.pickle","wb"))
# pickle.dump(test,open("test_spec_image.pickle","wb"))
# pickle.dump(train,open("train_spec_image.pickle","wb"),protocol=4)

In [ ]:
sample_spec = train[1]

In [ ]:
# plt.imshow(sample_spec[:,:,5])
plt.pcolormesh(sample_spec[:,:,5])
plt.show()

In [ ]:
!tree
del val,test,train